In [ ]:
# Formatting notebook to fit the browser size

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Loading libraries and packages


%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mtick
import scipy.optimize
import scipy.stats
import scipy.integrate as integrate
import scipy.integrate as special
import pandas as pd
import os
import linecache
from statistics import mean

plt.style.use('ggplot') # use ggplot style for graps

In [ ]:
# Import data

# setup path of input file
input_data_file = os.path.join('data_inputs', filename) #note - this uses a variable called file name as the input file, if you are only running one file you can change this to the name of your file in quotation marks 

#getting the base name of the file which will be used to name the output file in the same way
name = ((os.path.splitext(input_data_file))[0]).lstrip('data_inputs\\') #This takes the file name, splits it at the extension (e.g. csv), then takes the first element of this and removes the data_inputs\ folder location from it, this results in the file name

# define column headings for dataframe
headings_list = ['Bin_number','Bin_diameter_lower', 'Diff_volume', 'Diff_number_perc', 'Diff_number_gram', 'Diff_surface_area', 'Diff_number']

# read in dataframe using pandas library 
input_data_df = pd.read_csv(input_data_file, sep='\,', engine='python', skiprows=55,
                            names=headings_list)

# calculate bin width in new column
input_data_df['Bin_width'] = input_data_df['Bin_diameter_lower'].shift(-1) - input_data_df['Bin_diameter_lower']

# read off the final bin number into a variable, called imput_data_bin_max, and then drop this row
input_data_bin_max = input_data_df.iloc[-1,1]
input_data_df = input_data_df.drop(input_data_df.index[input_data_df.tail(1).index])

# set 'Bin_number' column to be the index
input_data_df = input_data_df.set_index('Bin_number')
# change datatype of index from float to integer
input_data_df.index = input_data_df.index.astype('int')

# Calculate mid-point of bins
input_data_df['Bin_midpoint'] = input_data_df['Bin_diameter_lower']+input_data_df['Bin_width']/2

# Calc Diff_volume_density using: Diff_volume/Bin_width
input_data_df['Diff_volume_density'] = input_data_df['Diff_volume']/input_data_df['Bin_width']

# convert input data to lists
bin_midpoint_input_list = input_data_df['Bin_midpoint'].tolist()
diff_volume_density_input_list = input_data_df['Diff_volume_density'].tolist()

In [ ]:
# NORMAL FUNCTION

In [ ]:
# Define the normal functions
def normal_func(x_, A_2_, mu_2_, sigma_2_):
    return (  (A_2_/(sigma_2_*np.sqrt(2.*np.pi)))*np.exp( -( (x_ - mu_2_)/sigma_2_ )**2/2. )  )
            

In [ ]:
# Define a function to fit the normal function

def func_normal_fit(bin_midpoint_input_, diff_volume_density_input_):
    normal_initial_params_ = [10, 5 , 2.2]
    fit_params_normal, fit_cov_normal = scipy.optimize.curve_fit(normal_func,bin_midpoint_input_, diff_volume_density_input_, 
                                                    p0 = normal_initial_params_, bounds = ([0,0,0], [np.inf,np.inf,np.inf]))
    fit_params_err_normal = np.sqrt(np.diag(fit_cov_normal))
    return fit_params_normal, fit_params_err_normal

#This function will return: {list of fit parameters}, {list of fit uncertanties}

In [ ]:
# #Plots the initial parameters to visualise how close the initial parameters fit
# #This can be useful if your curves are very different from normal and the initial parameters need to be adjusted
# #Commented out so not produced when running in batch mode

# #Initial parameter estimates -  ensure these are the same as shown above
# normal_initial_params_ = [10, 5 , 2.2]

# # define x_values to plot
# x_vals_plot = np.arange(0.5,50,0.5)

# # define curves to fit
# normal_vals_curve = [normal_func(i_, normal_initial_params_[0], normal_initial_params_[1],
#                                        normal_initial_params_[2]) for i_ in x_vals_plot]


# #plot graphs of initial parameters
# fig, ax = plt.subplots(1,1,figsize=(15,8))
# for spine in ['left','right','top','bottom']:
#     ax.spines[spine].set_color('k')
# ax.set_facecolor('white')
# ax.plot(input_data_df['Bin_midpoint'].tolist(), input_data_df['Diff_volume_density'].tolist(),
#         linewidth=2, color='k',label='Data' )
# ax.plot(x_vals_plot, normal_vals_curve,
#        linewidth=2, color='b',label='Normal curve' )
# fig.suptitle('Initial parameter guess: '+name, fontsize=16)
# ax.set_xlabel("Diameter ($\mu $m)", fontsize=18)
# ax.xaxis.set_major_locator(plt.MaxNLocator(6))
# ax.set_ylabel("Volume (%)", fontsize=18)
# ax.tick_params(axis='both', which='major', labelsize=16)
# leg_00 = ax.legend(fontsize=16, loc='upper right', ncol=1,facecolor='white', framealpha=1)
# leg_00.get_frame().set_edgecolor('k')
# plt.tight_layout() 
# fig.subplots_adjust(top=0.9)

In [ ]:
# Run the normal fit 

#create empty lists which I will store failed/successful fittings in
failed_normal = []
successful_normal = []
fitting_failed_normal = []


#run the normal fit with the exception catcher
try:
    output_normal_fit_params_list, output_normal_fit_err = func_normal_fit(bin_midpoint_input_list, diff_volume_density_input_list)

except RuntimeError:
    failed_normal = name.split('\n') #we need the \n if not we get \n included in the sample names when we make a list of samples in batch mode

    #this sets the parameters to fitting failed so in the output file we get this message
    granule_diameter_normal = "fitting failed"
    granule_content_normal = "fitting failed"
    normal_standard_error_of_regression = "fitting failed"
    normal_uncertainity = "fitting failed"
    
    #this is used in an if statement during plotting so that if failed we don't get a graph
    fitting_failed_normal = "yes"

else:
    successful_normal = name.split('\n') #we need the \n if not we get \n included in the sample names when we make a list of samples in batch mode

      
    #Getting the parameters from the models
    #norm mean is equal to mu
    granule_diameter_normal = output_normal_fit_params_list[1] 
    
    #areas under curves - witht the scipy integrate function, the noraml functions have been redefined using the output parameters from the fit 
    #and the integrate function is used, the output is a tuple with an estimated value of the integral first and the second value is an upper bound on the error   
    def normal_func_fitting(x_):
        return (  (output_normal_fit_params_list[0]/(output_normal_fit_params_list[2]*np.sqrt(2.*np.pi)))*np.exp( -( (x_ - output_normal_fit_params_list[1])/output_normal_fit_params_list[2])**2/2. )  )    
    granule_content_normal = scipy.integrate.quad(normal_func_fitting,0, np.inf) #integrate between 0 and infinity
     
    #reduced uncertainity
    normal_uncertainity = np.sum(output_normal_fit_err)
    
    #standard error of regression
    normal_vals_list = [normal_func(input_data_df['Bin_midpoint'].tolist()[i_], *output_normal_fit_params_list) #the star takes all the values from output_fit_params and puts it in here
                        for i_ in range(len(input_data_df['Diff_volume_density']))]
    normal_standard_error_of_regression = np.sqrt( (1/len(input_data_df['Diff_volume_density']-2)) * (sum((np.array(input_data_df['Diff_volume_density'].tolist()) - np.array(normal_vals_list))**2) / sum((np.array( np.array(input_data_df['Bin_midpoint'].tolist()) - mean(input_data_df['Bin_midpoint'].tolist()) ))**2)) )

In [ ]:
# # Plot optimized fit of normal fitting (if it was successful) 
# # Commented out so not produced when running in batch mode

# if fitting_failed_normal == "yes":
#     pass
# else:
#     # define x_values to plot
#     x_vals_plot = np.arange(0.5,50,0.05)

#     # define curves to fit
#     normal_vals_curve = [normal_func(i_, *output_normal_fit_params_list) for i_ in x_vals_plot]
    

#     #plot
#     fig, ax = plt.subplots(1,1,figsize=(15,8))
#     for spine in ['left','right','top','bottom']:
#            ax.spines[spine].set_color('k')
#     ax.set_facecolor('white')
#     ax.plot(input_data_df['Bin_midpoint'].tolist(), input_data_df['Diff_volume_density'].tolist(),
#             linewidth=2, color='k',label='Data' )
#     ax.plot(x_vals_plot, normal_vals_curve,
#             linewidth=2, color='b',label='Fitted Normal Curve\nTotal uncertainty = %.2f\nStandard error of regression = %.4f' % (normal_uncertainity, normal_standard_error_of_regression)  )
#     fig.suptitle('Normal fit: '+name, fontsize=16)
#     ax.set_xlabel("Diameter ($\mu $m)", fontsize=18)
#     ax.xaxis.set_major_locator(plt.MaxNLocator(6))
#     ax.set_ylabel("Volume (%)", fontsize=18)
#     ax.tick_params(axis='both', which='major', labelsize=16)
#     leg_00 = ax.legend(fontsize=16, loc='upper right', ncol=1,facecolor='white', framealpha=1)
#     leg_00.get_frame().set_edgecolor('k')
#     plt.tight_layout()
#     fig.subplots_adjust(top=0.9)

# #Saving plots in a folder called output - so make sure you have one in your working directory
# #if you want to change the folder name you just need to change the 'output' to your folder name, or you can remove it entirely
# plot_name = (name.strip('\n')+"_output.pdf")
# fig.savefig('output/'+name+".pdf", bbox_inches = 'tight', dpi=300)

In [ ]:
# LOGNORMAL FUNCTION

In [ ]:
# Define lognormal function

def lognormal_func(x_, A_1_, mu_1_, sigma_1_):
    return (  (A_1_/(x_*sigma_1_*np.sqrt(2.*np.pi)))*np.exp( -((np.log(x_)-mu_1_)/sigma_1_)**2/2. )  )

In [ ]:
# Define a function to fit the lognormal function

def func_lognormal_fit(bin_midpoint_input_, diff_volume_density_input_):
    lognormal_initial_params_ = [25, 1.8 , 0.4]
    fit_params_lognormal, fit_cov_lognormal = scipy.optimize.curve_fit(lognormal_func,bin_midpoint_input_, diff_volume_density_input_, 
                                                    p0 = lognormal_initial_params_, bounds = ([0,0,0], [np.inf,np.inf,np.inf]))
    fit_params_err_lognormal = np.sqrt(np.diag(fit_cov_lognormal))
    return fit_params_lognormal, fit_params_err_lognormal

#This function will return: {list of fit parameters}, {list of fit uncertanties}

In [ ]:
# #Plots the initial parameters to visualise how close the initial parameters fit
# #This can be useful if your curves are very different from normal and the initial parameters need to be adjusted
# #Commented out so not produced when running in batch mode

# #Initial parameter estimates -  ensure these are the same as shown above
# lognormal_initial_params_ = [25, 1.8 , 0.4]

# # define x_values to plot
# x_vals_plot = np.arange(0.5,50,0.5)

# # define curves to fit
# lognormal_vals_curve = [lognormal_func(i_, lognormal_initial_params_[0], lognormal_initial_params_[1],
#                                        lognormal_initial_params_[2]) for i_ in x_vals_plot]


# #plot graphs of initial parameters
# fig, ax = plt.subplots(1,1,figsize=(15,8))
# for spine in ['left','right','top','bottom']:
#     ax.spines[spine].set_color('k')
# ax.set_facecolor('white')
# ax.plot(input_data_df['Bin_midpoint'].tolist(), input_data_df['Diff_volume_density'].tolist(),
#         linewidth=2, color='k',label='Data' )
# ax.plot(x_vals_plot, lognormal_vals_curve,
#        linewidth=2, color='b',label='Lognormal curve' )
# fig.suptitle('Initial parameter guess: '+name, fontsize=16)
# ax.set_xlabel("Diameter ($\mu $m)", fontsize=18)
# ax.xaxis.set_major_locator(plt.MaxNLocator(6))
# ax.set_ylabel("Volume (%)", fontsize=18)
# ax.tick_params(axis='both', which='major', labelsize=16)
# leg_00 = ax.legend(fontsize=16, loc='upper right', ncol=1,facecolor='white', framealpha=1)
# leg_00.get_frame().set_edgecolor('k')
# plt.tight_layout() 
# fig.subplots_adjust(top=0.9)

In [ ]:
# Run the lognormal fit 

#create empty lists which I will store failed/successful fittings in
failed_lognormal = []
successful_lognormal = []
fitting_failed_lognormal = []


#run the lognormal fit with the exception catcher
try:
    output_lognormal_fit_params_list, output_lognormal_fit_err = func_lognormal_fit(bin_midpoint_input_list, diff_volume_density_input_list)

except RuntimeError:
    failed_lognormal = name.split('\n') #we need the \n if not we get \n included in the sample names when we make a list of samples in batch mode

    #this sets the parameters to fitting failed so in the output file we get this message
    granule_diameter_lognormal = "fitting failed"
    granule_content_lognormal = "fitting failed"
    lognormal_standard_error_of_regression = "fitting failed"
    lognormal_uncertainity = "fitting failed"
    
    #this is used in an if statement during plotting so that if failed we don't get a graph
    fitting_failed_lognormal = "yes"

else:
    successful_lognormal = name.split('\n') #we need the \n if not we get \n included in the sample names when we make a list of samples in batch mode

      
    #Getting the parameters from the models
    #lognorm mean is equal to exp(mu + (sigma^2/2))
    granule_diameter_lognormal = np.exp(output_lognormal_fit_params_list[1] + ((output_lognormal_fit_params_list[2]**2)/2))
    
    #areas under curves - witht the scipy integrate function, the noraml functions have been redefined using the output parameters from the fit 
    #and the integrate function is used, the output is a tuple with an estimated value of the integral first and the second value is an upper bound on the error
    def lognormal_func_fitting(x_):
        return (output_lognormal_fit_params_list[0]/(x_*output_lognormal_fit_params_list[2]*np.sqrt(2.*np.pi)))*np.exp( -((np.log(x_)-output_lognormal_fit_params_list[1])/output_lognormal_fit_params_list[2])**2/2. )
    granule_content_lognormal = scipy.integrate.quad(lognormal_func_fitting,0, np.inf)  #integrate between 0 and infinity
    
    #reduced uncertainity
    lognormal_uncertainity = np.sum(output_lognormal_fit_err)
    
    #standard error of regression
    lognormal_vals_list = [lognormal_func(input_data_df['Bin_midpoint'].tolist()[i_], *output_lognormal_fit_params_list) #the star takes all the values from output_fit_params and puts it in here
                        for i_ in range(len(input_data_df['Diff_volume_density']))]
    lognormal_standard_error_of_regression = np.sqrt( (1/len(input_data_df['Diff_volume_density']-2)) * (sum((np.array(input_data_df['Diff_volume_density'].tolist()) - np.array(lognormal_vals_list))**2) / sum((np.array( np.array(input_data_df['Bin_midpoint'].tolist()) - mean(input_data_df['Bin_midpoint'].tolist()) ))**2)) )

In [ ]:
# # Plot optimized fit of lognormal fitting (if it was successful) 
# # Commented out so not produced when running in batch mode

# if fitting_failed_lognormal == "yes":
#     pass
# else:
#     # define x_values to plot
#     x_vals_plot = np.arange(0.5,50,0.05)

#     # define curves to fit
#     lognormal_vals_curve = [lognormal_func(i_, *output_lognormal_fit_params_list) for i_ in x_vals_plot]
    

#     #plot
#     fig, ax = plt.subplots(1,1,figsize=(15,8))
#     for spine in ['left','right','top','bottom']:
#            ax.spines[spine].set_color('k')
#     ax.set_facecolor('white')
#     ax.plot(input_data_df['Bin_midpoint'].tolist(), input_data_df['Diff_volume_density'].tolist(),
#             linewidth=2, color='k',label='Data' )
#     ax.plot(x_vals_plot, lognormal_vals_curve,
#             linewidth=2, color='b',label='Fitted Lognormal Curve\nTotal uncertainty = %.2f\nStandard error of regression = %.4f' % (lognormal_uncertainity, lognormal_standard_error_of_regression)  )
#     fig.suptitle('Lognormal fit: '+name, fontsize=16)
#     ax.set_xlabel("Diameter ($\mu $m)", fontsize=18)
#     ax.xaxis.set_major_locator(plt.MaxNLocator(6))
#     ax.set_ylabel("Volume (%)", fontsize=18)
#     ax.tick_params(axis='both', which='major', labelsize=16)
#     leg_00 = ax.legend(fontsize=16, loc='upper right', ncol=1,facecolor='white', framealpha=1)
#     leg_00.get_frame().set_edgecolor('k')
#     plt.tight_layout()
#     fig.subplots_adjust(top=0.9)

# #Saving plots in a folder called output - so make sure you have one in your working directory
# #if you want to change the folder name you just need to change the 'output' to your folder name, or you can remove it entirely
# plot_name = (name.strip('\n')+"_output.pdf")
# fig.savefig('output/'+name+".pdf", bbox_inches = 'tight', dpi=300)

In [ ]:
# COMBINING THE OUTPUT OF THE DIFFERENT FITS

In [ ]:
# # Plotting all fits 
# # Commented out so not produced when running in batch mode

# #defining x values
# x_vals_plot = np.arange(0.5,50,0.05)
# fig, ax = plt.subplots(1,1,figsize=(15,8))
# for spine in ['left','right','top','bottom']:
#     ax.spines[spine].set_color('k')
# ax.set_facecolor('white')
# ax.plot(input_data_df['Bin_midpoint'].tolist(), input_data_df['Diff_volume_density'].tolist(),
#         linewidth=2, color='k',label='\nData\n' )
# if fitting_failed_normal == "yes":
#     pass
# else:
#     ax.plot(x_vals_plot, normal_vals_curve, 
#         linewidth=2, color='g',label='Normal\nStandard error of regression =%.4f\n' % (normal_standard_error_of_regression))  
# if fitting_failed_lognormal == "yes":
#     pass
# else:
#     ax.plot(x_vals_plot, lognormal_vals_curve,
#         linewidth=2, color='b',label='Lognormal\nStandard error of regression =%.4f\n' % (lognormal_standard_error_of_regression))
# ax.set_xlabel("Diameter ($\mu $m)", fontsize=18)
# ax.xaxis.set_major_locator(plt.MaxNLocator(6))
# ax.set_ylabel("Volume (%)", fontsize=18)
# ax.tick_params(axis='both', which='major', labelsize=16)
# ax.set_title('All fits', fontsize=18)
# leg_00 = ax.legend(fontsize=16, loc='upper right', ncol=1,facecolor='white', framealpha=1)
# leg_00.get_frame().set_edgecolor('k')

In [ ]:
# Combining all plots together in one pdf file
# Note - the if-else loops are used so that if the fitting failed then the graph is blank and says fitting failed
# Note - saves the pdf in a folder called output

#defining x values
x_vals_plot = np.arange(0.5,50,0.05)

fig, ax = plt.subplots(3,1,figsize=(15,25))
fig.suptitle(name, fontsize=20, y=1.01, fontweight="bold")
fig.subplots_adjust(top=0.95)
fig.tight_layout(h_pad=8)

#first graph - normal graph
normal_vals_curve = [normal_func(i_, *output_normal_fit_params_list) 
                        for i_ in x_vals_plot]
for spine in ['left','right','top','bottom']:
    ax[0].spines[spine].set_color('k')
ax[0].set_facecolor('white')
if fitting_failed_normal == "yes":
    ax[0].text(0.4, 0.5, 'Normal fitting failed', fontsize = 16)
else:
    ax[0].plot(input_data_df['Bin_midpoint'].tolist(), input_data_df['Diff_volume_density'].tolist(),
            linewidth=2, color='k',label='Data' )
    ax[0].plot(x_vals_plot, normal_vals_curve,
            linewidth=2, color='b',label='Normal'  )
    ax[0].plot(x_vals_plot, normal_vals_curve,
               linewidth=2, color='w', alpha=0, label='Total uncertainty = %.2f\nStandard error of regression =%.4f' %
           (normal_uncertainity, normal_standard_error_of_regression))
    leg_00 = ax[0].legend(fontsize=12, loc='upper right', ncol=1,facecolor='white', framealpha=1)
    leg_00.get_frame().set_edgecolor('k')

ax[0].set_xlabel("Diameter ($\mu $m)", fontsize=12)
ax[0].xaxis.set_major_locator(plt.MaxNLocator(6))
ax[0].set_ylabel("Volume (%)", fontsize=12)
ax[0].tick_params(axis='both', which='major', labelsize=12)
ax[0].set_title('Normal', fontsize=12)

#second graph - lognormal graph
lognormal_vals_curve = [lognormal_func(i_, *output_lognormal_fit_params_list) 
                         for i_ in x_vals_plot]
for spine in ['left','right','top','bottom']:
        ax[1].spines[spine].set_color('k')
        ax[1].set_facecolor('white')
if fitting_failed_lognormal == "yes":
    ax[1].text(0.4, 0.5, 'Lognormal fitting failed', fontsize = 16)
else:
    ax[1].plot(input_data_df['Bin_midpoint'].tolist(), input_data_df['Diff_volume_density'].tolist(),
            linewidth=2, color='k',label='Data' )
    ax[1].plot(x_vals_plot, lognormal_vals_curve,
            linewidth=2, color='b',label='Lognormal' )
    ax[1].plot(x_vals_plot, normal_vals_curve,
            linewidth=2, color='w', alpha=0, label='Total uncertainty = %.2f\nStandard error of regression =%.4f' %
           (lognormal_uncertainity, lognormal_standard_error_of_regression))
    leg_01 = ax[1].legend(fontsize=12, loc='upper right', ncol=1,facecolor='white', framealpha=1)
    leg_01.get_frame().set_edgecolor('k')
    
ax[1].set_ylabel("Volume (%)", fontsize=18)
ax[1].set_xlabel("Diameter ($\mu $m)", fontsize=12)
ax[1].set_title('Lognormal', fontsize=12)
ax[1].xaxis.set_major_locator(plt.MaxNLocator(6))
ax[1].tick_params(axis='both', which='major', labelsize=12)


#fourth graph - all fits combined on one graph
for spine in ['left','right','top','bottom']:
    ax[2].spines[spine].set_color('k')
ax[2].set_facecolor('white')
ax[2].plot(input_data_df['Bin_midpoint'].tolist(), input_data_df['Diff_volume_density'].tolist(),
        linewidth=2, color='k',label='\nData\n' )
if fitting_failed_normal == "yes":
    pass
else:
    ax[2].plot(x_vals_plot, normal_vals_curve, 
        linewidth=2, color='g',label='Normal\nStandard error of regression =%.4f\n' % (normal_standard_error_of_regression))  
if fitting_failed_lognormal == "yes":
    pass
else:
    ax[2].plot(x_vals_plot, lognormal_vals_curve,
        linewidth=2, color='b',label='Lognormal\nStandard error of regression =%.4f\n' % (lognormal_standard_error_of_regression))
ax[2].set_xlabel("Diameter ($\mu $m)", fontsize=12)
ax[2].xaxis.set_major_locator(plt.MaxNLocator(6))
ax[2].set_ylabel("Volume (%)", fontsize=12)
ax[2].tick_params(axis='both', which='major', labelsize=12)
ax[2].set_title('All fits', fontsize=12)
leg_02 = ax[2].legend(fontsize=12, loc='upper right', ncol=1,facecolor='white', framealpha=0.5)
leg_02.get_frame().set_edgecolor('k')


#Saving plots in a folder called output
plot_name = (name.strip('\n')+"_output.pdf")
fig.savefig('output/'+name+".pdf", bbox_inches = 'tight', dpi=300)

In [ ]:
#Getting the parameters from the different fits
normal_parameters = (granule_diameter_normal, granule_content_normal[0], normal_uncertainity, normal_standard_error_of_regression,)
lognormal_parameters = (granule_diameter_lognormal, granule_content_lognormal[0], lognormal_uncertainity, lognormal_standard_error_of_regression)

#Joining together the sample name and fitting parameters
name_str = (name,) #converts the sample name into a tuple so it is in the same format as the parameters
fitting_parameters = (name_str + normal_parameters + lognormal_parameters)